In [403]:
import os
import sys
import random
import torch
import numpy as np
from sklearn.utils import shuffle
import torch.nn.functional as F

In [2]:
pwd

'/home/andrew/Desktop/CS330/hw1'

In [3]:
# Train Omniglot dataset
from load_data import get_images, image_file_to_array

In [1163]:
# Change class DataGenerator
class DataGenerator(object):
    """
    Data Generator capable of generating batches of Omniglot data.
    A "class" is considered a class of omniglot digits.
    """

    def __init__(self, num_classes, num_samples_per_class, config={}, device = torch.device('cuda')):
        """
        Args:
            num_classes: int
                Number of classes for classification (N-way)
            
            num_samples_per_class: int
                Number of samples per class in the support set (K-shot).
                Will generate additional sample for the querry set.
                
            device: cuda.device: 
                Device to allocate tensors to.
        """
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes

        data_folder = config.get('data_folder', './data/omniglot_resized')
        self.img_size = config.get('img_size', (28, 28))

        self.dim_input = np.prod(self.img_size)
        self.dim_output = self.num_classes

        character_folders = [os.path.join(data_folder, family, character)
                             for family in os.listdir(data_folder)
                             if os.path.isdir(os.path.join(data_folder, family))
                             for character in os.listdir(os.path.join(data_folder, family))
                             if os.path.isdir(os.path.join(data_folder, family, character))]

        random.seed(1)
        random.shuffle(character_folders)
        num_val = 100
        num_train = 1100
        self.metatrain_character_folders = character_folders[: num_train]
        self.metaval_character_folders = character_folders[
            num_train:num_train + num_val]
        self.metatest_character_folders = character_folders[
            num_train + num_val:]
        self.device = device

    def sample_batch(self, batch_type, batch_size):
        """
        Samples a batch for training, validation, or testing
        Args:
            batch_type: str
                train/val/test set to sample from
                
            batch_size: int:
                Size of batch of tasks to sample
                
        Returns:
            images: tensor
                A tensor of images of size [B, K+1, N, 784]
                where B is batch size, K is number of samples per class, 
                N is number of classes
                
            labels: tensor
                A tensor of images of size [B, K+1, N, N] 
                where B is batch size, K is number of samples per class, 
                N is number of classes
        """
        if batch_type == "train":
            folders = self.metatrain_character_folders
        elif batch_type == "val":
            folders = self.metaval_character_folders
        else:
            folders = self.metatest_character_folders

        #############################
        #### YOUR CODE GOES HERE ####
        #############################
        
        # Sample N different character and labels from train, test, validation
        B = batch_size
        N = self.num_classes
        K = self.num_samples_per_class
        dim = self.dim_input 
        batch_images = []
        batch_labels = []
        
        # Pick number of task equal to batch
        for i in range(B):
            # Sample from folder with selected number of class
            sampled_class = random.sample(folders, N)
            
            # Load K+1 images per char and collect labels, using K images per class for support set and one image per class for the query class
            # Create label matrix of size N*N using identity matrix, since for each class will have it own correspondence label encoded
            labels_encoded = np.identity(N)
            #print(labels_encoded)
            
            # Collect image and labels with K+1 sample for each sampeld class, have shape
            labels_imgs = get_images(sampled_class, labels_encoded, K+1, shuffle=False) # N * (K_+ 1) 
            
            # Create tensor and load data in support and train
            #labels_imgs_matrix = np.reshape(labels_imgs, (K + 1, N, 784))
            
            support_set = [] # K * N * dim
            query_set = [] # 1 * N * dim
            support_set_label = [] 
            query_set_label = [] 
            
            # query will have shape 1 * N * dim
            # support will have shape K * N * dim
            # take first sample of each character batch for the query set
            test_counter = 0
            for j in range(len(labels_imgs)): 
                if j == test_counter:
                    query_set.append(image_file_to_array(labels_imgs[j][1], dim)) 
                    query_set_label.append(labels_imgs[j][0])
                    #print(labels_imgs[j][1]) # path
                    #print(labels_imgs[j][0]) # label
                    test_counter += (K+1)
                    # print(i, j), 0, 6, 12
                else:
                    support_set.append(image_file_to_array(labels_imgs[j][1], dim))
                    support_set_label.append(labels_imgs[j][0])
                    #print(labels_imgs[j][1])
                    #print(labels_imgs[j][0])
            
            
            # Shuffle query set only
            #print(query_set_label)
            print(np.asarray(query_set_label).shape)
            query_set, query_set_label = shuffle(query_set, query_set_label)
            #print(query_set_label)
            #support_set, support_set_label = shuffle(support_set, support_set_label)
            #print(np.asarray(query_set).shape)    1 * N
            print(np.asarray(support_set_label).shape)  #K * N
            
            # Put to images tensor (K + 1) * N * dim 
            images_matrix = np.concatenate((support_set, query_set), axis=0)
            # print(images_matrix.shape)
            images_matrix = images_matrix.reshape((K + 1, N, dim))
            #print(images_matrix.shape)
            
            # Put to labels tensor (K + 1) * N * N
            labels_matrix = np.concatenate((support_set_label, query_set_label), axis=0)
            #print(labels_matrix.shape)
            #print(labels_matrix[len(labels_matrix)-1].shape)
            #print(labels_matrix[len(labels_matrix)-1])
            #print(labels_matrix[len(labels_matrix)-2])  # this query set
            #print(labels_matrix[len(labels_matrix)-3])
            labels_matrix[len(labels_matrix)-3][2] = 7
            labels_matrix[len(labels_matrix)-2][2] = 8
            labels_matrix[len(labels_matrix)-1][2] = 9
            #print(labels_matrix[len(labels_matrix)-1])
            #print(labels_matrix[len(labels_matrix)-2])  # this query set
            #print(labels_matrix[len(labels_matrix)-3])
                  
            labels_matrix = labels_matrix.reshape((K + 1, N, N))
            #print('Query')
            print(labels_matrix[K][0])
            print(labels_matrix[K][1])
            print(labels_matrix[K][2])
            #print(labels_matrix.shape)
            #print('End')
            
            # Add to batch
            batch_images.append(images_matrix)
            batch_labels.append(labels_matrix)
            
        #print(np.asarray(batch_images).shape)
        #print(np.asarray(batch_labels).shape)
        
        # SOLUTION:
        # Format the data and return two matrices, one of flattened images with specified shape
        return torch.FloatTensor(batch_images), torch.FloatTensor(batch_labels)

In [1164]:
# n = 3 classes
# k = 5 ways
test = DataGenerator(3, 5)

In [1165]:
a = test.sample_batch("train", 2)

(3, 3)
(15, 3)
[0. 1. 7.]
[1. 0. 8.]
[0. 0. 9.]
(3, 3)
(15, 3)
[0. 1. 7.]
[0. 0. 8.]
[1. 0. 9.]


In [1127]:
a[1].shape

torch.Size([2, 6, 3, 3])

In [1131]:
# first batch, last K + 1, first char 
a[1][0][5][0]

tensor([1., 0., 7.])

In [1132]:
# first batch, last K + 1, second char 
a[1][0][5][1]

tensor([0., 0., 8.])

In [1134]:
# second batch, last K + 1, second char 
a[1][1][5][1]

tensor([1., 0., 8.])

In [1139]:
b = a[1].reshape(2*6*3, 3)

In [1141]:
b.shape

torch.Size([36, 3])

In [1149]:
# query, first batch, 18, 18/3 class = 6 => index 5, 11, 17
# oh no, it will be input at the bottom of the matrix
b[15]

tensor([1., 0., 7.])

In [1148]:
b[16]

tensor([0., 0., 8.])

In [1150]:
b[17]

tensor([0., 1., 9.])

In [1152]:
b[35]

tensor([0., 1., 9.])

In [1153]:
b[34]

tensor([1., 0., 8.])

In [1154]:
b[33]

tensor([0., 0., 7.])

In [1118]:
a = test.sample_batch("train", 3)

[1. 0. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 9.]
[0. 0. 8.]
[0. 1. 7.]
Query
[0. 1. 7.]
[0. 0. 8.]
[1. 0. 9.]
End
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[1. 0. 9.]
[0. 1. 8.]
[0. 0. 7.]
Query
[0. 0. 7.]
[0. 1. 8.]
[1. 0. 9.]
End
[1. 0. 0.]
[0. 0. 1.]
[0. 1. 0.]
[1. 0. 9.]
[0. 0. 8.]
[0. 1. 7.]
Query
[0. 1. 7.]
[0. 0. 8.]
[1. 0. 9.]
End


In [1114]:
# 4 classese, 10 shots, N = 4, K = 10
test = DataGenerator(4, 10)

In [1115]:
# 2 batches, output images B * (K + 1) * N * dim
# 2 batches, output batches B * (K + 1) * N * N
a = test.sample_batch("train", 3)

[0. 0. 0. 1.]
[0. 1. 0. 0.]
[1. 0. 0. 0.]
[0. 0. 9. 1.]
[0. 1. 8. 0.]
[1. 0. 7. 0.]
Query
[0. 0. 1. 0.]
[1. 0. 7. 0.]
[0. 1. 8. 0.]
End
[0. 0. 1. 0.]
[1. 0. 0. 0.]
[0. 1. 0. 0.]
[0. 0. 9. 0.]
[1. 0. 8. 0.]
[0. 1. 7. 0.]
Query
[0. 0. 0. 1.]
[0. 1. 7. 0.]
[1. 0. 8. 0.]
End
[0. 0. 1. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[0. 0. 9. 0.]
[1. 0. 8. 0.]
[0. 0. 7. 1.]
Query
[0. 1. 0. 0.]
[0. 0. 7. 1.]
[1. 0. 8. 0.]
End


In [410]:
# 2 batches, output images B * (K + 1) * N * dim
# 2 batches, output batches B * (K + 1) * N * N
a = test.sample_batch("train", 3)

(3, 11, 4, 784)
(3, 11, 4, 4)


In [164]:
type(a[0])

numpy.ndarray

In [165]:
from hw1 import *

In [1032]:
# Change class MANN
class MANN(nn.Module):

    def __init__(self, num_classes, samples_per_class, model_size=128, input_size=784):
        super(MANN, self).__init__()
        
        def initialize_weights(model):
            nn.init.xavier_uniform_(model.weight_ih_l0)
            nn.init.zeros_(model.bias_hh_l0)
            nn.init.zeros_(model.bias_ih_l0)
    
        self.num_classes = num_classes
        self.samples_per_class = samples_per_class
        self.input_size = input_size
        self.layer1 = torch.nn.LSTM(num_classes + input_size, 
                                    model_size, 
                                    batch_first=True)
        self.layer2 = torch.nn.LSTM(model_size,
                                    num_classes,
                                    batch_first=True)
        initialize_weights(self.layer1)
        initialize_weights(self.layer2)
        
        self.dnc = DNC(
                       input_size=num_classes + input_size,
                       output_size=num_classes,
                       hidden_size=model_size,
                       rnn_type='lstm',
                       num_layers=1,
                       num_hidden_layers=1,
                       nr_cells=num_classes,
                       cell_size=64,
                       read_heads=1,
                       batch_first=True,
                       gpu_id=0,
                       )

    def forward(self, input_images, input_labels):
        """
        MANN
        Args:
            input_images: tensor
                A tensor of shape [B, K+1, N, 784] of flattened images
            
            labels: tensor:
                A tensor of shape [B, K+1, N, N] of ground truth labels
        Returns:
            
            out: tensor
            A tensor of shape [B, K+1, N, N] of class predictions
        """
        #############################
        #### YOUR CODE GOES HERE ####
        #############################

        # SOLUTION:
        N = self.num_classes
        K = self.samples_per_class
        dim = self.input_size
        B = len(input_images)
        
        # reshape tensors to process
        reshaped_image = torch.reshape(input_images, (B * N * (K+1), dim))
        reshaped_label = torch.reshape(input_labels, (B * N * (K+1), N))
        print(reshaped_image.shape) # (88, 784)
        print(reshaped_label.shape) # (88, 4)
        
        # conatenate query image with label of zeros
        # Notes: in the previous part, we put all the query set of each character in each batch to the bottom, for each batch in B and each characters in N,
        # the vectors at K + 1 position in [B, K + 1, N, 784] are the query set
        # E.g: n = 4 classes, k = 10 ways, batch = 2, we have output shape of image batch is (2, 11, 4, 784)
        # For first batch, if we resize the tensor to (1, 44, 784), the last three vectors of index 10, 21, 32, 43 are in the query set
        # And if we resize tensor with both batch to (2 * 44, 784) = (88, 784) the query set is in position (41, 42, 43, 44) and (85, 86, 87, 88)
        # That are the indexes that we need to concatenate with vector of zeros, we can start to change the labels of these indexes before concatenating
        for i in range(len(reshaped_label)):
            # find last set of batch label, for e.g. example above, index should be 40, 41, 42, 43 and 84, 85, 86, 87
            if (i + 1) % (N * (K + 1)) == 0:
                #print(i)
                for j in range(i, i - N, -1):
                    #print(j)
                    #print(reshaped_label[j])
                    reshaped_label[j] = torch.zeros(N)
                    #print(reshaped_label[j])
        
        # concatenate image and label
        concatenated = torch.cat((reshaped_image, reshaped_label), dim = 1)
        #print(concatenated.shape)
        print(concatenated[j][783:]) # should be vector of 0 * N
        
        concatenated = torch.reshape(concatenated, (B, (K+1) * N, dim + N))
        concatenated = concatenated.to(torch.double)
        #print(concatenated.shape)
        #print(type(concatenated))
        #print(concatenated.dtype)
        # Check if label changed to zero
        print('Check label batch 1, N * label at the end of batch should be zero * N')
        print(concatenated[0].size()) # 44 * 788
        print(concatenated[0][10].size()) # 788
        print(concatenated[0][40][783:]) # label should be zero 
        print(concatenated[0][41][783:]) 
        print(concatenated[0][42][783:]) 
        print(concatenated[0][43][783:]) 
        
        output, _ = self.layer1(concatenated)
        #print(type(output))
        #print(output.size())
        output, _ = self.layer2(output)
        #print(output.size())
        output = torch.reshape(output, (B, K + 1, N, N))
        #print(output.shape)
        output = output.to(torch.float)
        
        return output


    def loss_function(self, preds, labels):
        """
        Computes MANN loss
        Args:
            preds: tensor
                A tensor of shape [B, K+1, N, N] of network outputs
            
            labels: tensor
                A tensor of shape [B, K+1, N, N] of class labels
                
        Returns:
            scalar loss
        """
        #############################
        #### YOUR CODE GOES HERE ####
        #############################

        # SOLUTION: 
        
        N = self.num_classes
        K = self.samples_per_class
        dim = self.input_size
        B = len(labels)
        
        # Reshape two inputs into [B * (K+1) * N, N]
        reshaped_preds = torch.reshape(preds, (B * (K+1) * N, N))
        reshaped_labels = torch.reshape(labels, (B * (K+1) * N, N))
        print(reshaped_preds.size())
        print(reshaped_labels.size())
        #return reshaped_preds [88, 4]
        
        # Get prediction and label from the last items, should be  last N * 1 sample
        preds_N = reshaped_preds[(len(reshaped_preds) -  B*N):, :]
        labels_N = reshaped_labels[(len(reshaped_preds) -  B*N):, :]
        print(preds_N.shape) # 8 * 4 # one-hot encoding for the label
        print(labels_N.shape)
        # transfor to 8 * 1
        labels_N = torch.argmax(labels_N, dim = 1)
        preds_N = preds_N.to('cuda')
        print(labels_N.size())
        
        output = F.cross_entropy(preds_N, labels_N)

        return output 

In [1033]:
test2 = MANN(4, 10)

In [1006]:
# B * (K+1) * N * dim
a[0].shape

(2, 11, 4, 784)

In [1035]:
# B * (K+1) * N * N
a[1].shape

(2, 11, 4, 4)

In [1050]:
torch.from_numpy(a[0]).to('cuda').is_cuda

True

In [1041]:
test2 = test2.to(torch.double)
new = test2.forward(torch.from_numpy(a[0]), torch.from_numpy(a[1]))

torch.Size([88, 784])
torch.Size([88, 4])
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)
Check label batch 1, N * label at the end of batch should be zero * N
torch.Size([44, 788])
torch.Size([788])
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)


In [1045]:
new.shape

torch.Size([2, 11, 4, 4])

In [1046]:
new.is_cuda

False

In [1009]:
new.shape
new = torch.reshape(new, (2*11*4, 4))
new.shape

torch.Size([88, 4])

In [1010]:
new =torch.reshape(new, (2, 11, 4, 4))
new.shape

torch.Size([2, 11, 4, 4])

In [1011]:
new.shape # [B, K + 1, N, N]

torch.Size([2, 11, 4, 4])

In [1012]:
torch.from_numpy(a[1]).shape

torch.Size([2, 11, 4, 4])

In [1055]:
# [B * (K + 1,) * N, N] => 88 * 4
# choose last 4 query set: 4 * 4 (4 sample, each sample has label length of 4)
x = test2.loss_function(new.to('cuda'), torch.from_numpy(a[1]).to('cuda')) 

torch.Size([88, 4])
torch.Size([88, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8])


In [1056]:
x

tensor(1.3762, device='cuda:0', grad_fn=<NllLossBackward>)

In [1057]:
#!python3 hw1.py --num_classes=2 --num_samples=1 --meta_batch_size= 4

In [1166]:
1e-3

0.001